# Visualize quantile and parametric forecasts

In [ ]:
# imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from create_parametric_forecasts import ParametricForecasts

In [2]:
# Specify the forecast to investigate
path_quantile = 'C:\\Users\\fh6281\\PycharmProjects\\GermanBuildingDate\\02_forecast\\storage_quantile_fc\\file_fc_PatchTST_SFH3_2025-07-16_13-47-24_freq15.csv'

# get the corresponding path to the parametric forecast
path_parametric = path_quantile.replace('storage_quantile_fc', 'storage_parametric_fc').replace('.csv', '_parametric.csv')

In [3]:
# Define visualization functions
def plot_specific_day(df_fc, df_gt, time, title=None):

    fc_time = df_fc.loc[time]
    gt_time = df_gt.loc[time]

    fig = go.Figure()
    for col in fc_time.columns:
        fig.add_trace(go.Scatter(x=fc_time.index, y=fc_time[col], mode='lines', name=col))
    fig.add_trace(go.Scatter(x=fc_time.index, y=gt.loc[fc_time.index, 'P_TOT'], mode='lines', name='Ground Truth', line=dict(color='black', width=4)))
    fig.update_layout(title=title if title else f'Forecast for {time}', xaxis_title='Time', yaxis_title='Value')
    fig.show()

## Quantile Forecasts

In [4]:
# Load the quantile forecasts
fc_quantile = pd.read_csv(path_quantile, index_col=[2, 1], parse_dates=True)


gt = fc_quantile['P_TOT'].reset_index()
gt.index = gt['timestamp']
gt = gt.drop(columns=['timestamp', 'time_fc_created'])
gt = gt[~gt.index.duplicated(keep='first')]

# Drop the building and P-TOT columns from the quantile forecast
fc_quantile = fc_quantile.drop(columns=['building', 'P_TOT'])

In [5]:
time_to_plot = '2020-05-13 08:00:00'
plot_specific_day(fc_quantile, gt, time_to_plot, title=f'Unsorted Quantile Forecast for {time_to_plot}')

In [6]:
pf = ParametricForecasts()
pf.load_quantile_forecasts(path_quantile)

fc_quantile_sorted = pf.sort_quantiles()
plot_specific_day(fc_quantile_sorted, gt, time_to_plot, title=f'Sorted Quantile Forecast for {time_to_plot}')

Sorting quantiles in ascending order…


In [14]:
fc_quantile_smoothed_over_time = pf.smooth_quantiles_over_time(fc_quantile_sorted, window_size=5)
plot_specific_day(fc_quantile_smoothed_over_time, gt, time_to_plot, title=f'Smoothed Quantile Forecast for {time_to_plot}')

Smoothing each quantile over time with a rolling mean…


In [15]:
fc_quantile_smoothed = pf.smooth_quantiles_at_each_time(fc_quantile_sorted, window_size=5)
plot_specific_day(fc_quantile_smoothed, gt, time_to_plot, title=f'Smoothed Quantile Forecast at Each Time for {time_to_plot}')

Smoothing all quantiles at each time with a rolling mean…


## Parametric Forecasts